# Model

In [1]:
import xarray as xr
import numpy as np
import os
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import scipy.stats as stats
import netCDF4
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split

import sys
import random
import os
import xarray as xr
import numpy as np
import numpy.ma as ma
import pandas as pd
import math
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.patches as patches
import matplotlib.ticker as ticker
from matplotlib.path import Path
from mpl_toolkits.axes_grid1 import make_axes_locatable
from eofs.xarray import Eof
import time
import h5py
from scipy.signal import correlation_lags
from scipy.stats import pearsonr
from scipy.stats import linregress
from datetime import datetime
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

# No Aerosol Run

In [2]:
ds_daily = xr.open_dataset('/home/giantstep5/rjones98/meteorology/ESS569/ai_ready/ds_jja.nc')
ds_daily = ds_daily.isel(lon=slice(0, 92), lat=slice(0, 52))
ds_daily = ds_daily.fillna(0)

dataset_cnn = ds_daily

feature_name   = ['cape', 'precipitation']
output_name    = ['ltg']

indices_train_val = np.arange(0, 488)
idx_test = np.arange(488, 736)
idx_train, idx_val = train_test_split(indices_train_val, test_size=0.25, random_state=13)
zdim, _, ydim, xdim = ds_daily[feature_name].to_array().shape

train_dataset_cnn_X = dataset_cnn[feature_name].isel(time=idx_train)
val_dataset_cnn_X = dataset_cnn[feature_name].isel(time=idx_val)
test_dataset_cnn_X  = dataset_cnn[feature_name].isel(time=idx_test)
train_dataset_cnn_y = dataset_cnn[output_name].isel(time=idx_train)
val_dataset_cnn_y = dataset_cnn[output_name].isel(time=idx_val)
test_dataset_cnn_y  = dataset_cnn[output_name].isel(time=idx_test)

In [3]:
observed_max = float(train_dataset_cnn_y.ltg.max().values)

class PenalizedMSELoss(nn.Module):
    def __init__(self, max_value, penalty_weight=10):
        super(PenalizedMSELoss, self).__init__()
        self.max_value = max_value
        self.penalty_weight = penalty_weight
        self.mse = nn.MSELoss()

    def forward(self, predictions, targets):
        # Standard MSE loss
        mse_loss = self.mse(predictions, targets)
        
        # Compute penalty for exceeding the maximum value
        penalty = torch.clamp(predictions - self.max_value, min=0) ** 2
        penalty_loss = penalty.mean()
        
        # Combine MSE loss with penalty
        total_loss = mse_loss + self.penalty_weight * penalty_loss
        return total_loss

# Initialize the penalized loss function
#criterion = PenalizedMSELoss(max_value=observed_max, penalty_weight=10)

In [4]:
seed = 13
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)

# Define the PyTorch model
class SimpleCNN(nn.Module):
    def __init__(self, zdim, ydim, xdim):
        super(SimpleCNN, self).__init__()
        self.encoder0 = self.encoder_block(zdim, 32)
        self.encoder1 = self.encoder_block(32, 16)
        self.center = self.conv_block(16, 8)
        self.decoder1 = self.decoder_block(8, 16)
        self.decoder0 = self.decoder_block(16, 32)
        self.outputs = nn.Conv2d(32, 1, kernel_size=1, padding=0)

    def conv_block(self, in_channels, out_channels):
        return nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),  # Added BatchNorm2d
            nn.ReLU(inplace=True)
        )

    def encoder_block(self, in_channels, out_channels):
        return nn.Sequential(
            self.conv_block(in_channels, out_channels),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

    def decoder_block(self, in_channels, out_channels):
        return nn.Sequential(
            nn.ConvTranspose2d(in_channels, out_channels, kernel_size=2, stride=2),
            nn.BatchNorm2d(out_channels),  # Added BatchNorm2d
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        encoder0 = self.encoder0(x)
        encoder1 = self.encoder1(encoder0)
        center = self.center(encoder1)
        decoder1 = self.decoder1(center)
        decoder0 = self.decoder0(decoder1)
        outputs = self.outputs(decoder0)
        return outputs

# Initialize model, loss function, and optimizer
zdim, ydim, xdim = 2, 208, 300  # Adjust these based on your data
model = SimpleCNN(zdim, ydim, xdim).to('cpu')
criterion = PenalizedMSELoss(max_value=observed_max, penalty_weight=5)  # Adjusted penalty_weight
optimizer = optim.Adam(model.parameters(), lr=0.00001)  # Reduced learning rate
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3)


In [5]:
def normalize(x, m, s):
    return (x - m).groupby('time.day') / s

def unnormalize(x, m, s):
    return (x * s).groupby('time.day') + m

train_dataset_cnn_X_norm = (
    train_dataset_cnn_X - train_dataset_cnn_X.mean(dim={'time', 'lat', 'lon'})
) / train_dataset_cnn_X.std(dim={'time', 'lat', 'lon'})
train_dataset_cnn_X_norm = train_dataset_cnn_X_norm.fillna(0)

val_dataset_cnn_X_norm = (
    val_dataset_cnn_X - val_dataset_cnn_X.mean(dim={'time', 'lat', 'lon'})
) / val_dataset_cnn_X.std(dim={'time', 'lat', 'lon'})
val_dataset_cnn_X_norm = val_dataset_cnn_X_norm.fillna(0)

test_dataset_cnn_X_norm = (
    test_dataset_cnn_X - test_dataset_cnn_X.mean(dim={'time', 'lat', 'lon'})
) / test_dataset_cnn_X.std(dim={'time', 'lat', 'lon'})
test_dataset_cnn_X_norm = test_dataset_cnn_X_norm.fillna(0)

train_dataset_cnn_y_norm = (
    train_dataset_cnn_y - train_dataset_cnn_y.mean(dim={'time', 'lat', 'lon'})
) / train_dataset_cnn_y.std(dim={'time', 'lat', 'lon'})
train_dataset_cnn_y_norm = train_dataset_cnn_y_norm.fillna(0)

val_dataset_cnn_y_norm = (
    val_dataset_cnn_y - val_dataset_cnn_y.mean(dim={'time', 'lat', 'lon'})
) / val_dataset_cnn_y.std(dim={'time', 'lat', 'lon'})
val_dataset_cnn_y_norm = val_dataset_cnn_y_norm.fillna(0)

test_dataset_cnn_y_norm = (
    test_dataset_cnn_y - test_dataset_cnn_y.mean(dim={'time', 'lat', 'lon'})
) / test_dataset_cnn_y.std(dim={'time', 'lat', 'lon'})
test_dataset_cnn_y_norm = test_dataset_cnn_y_norm.fillna(0)

# Convert to numpy arrays and move axis
tr_X = np.moveaxis(np.array(train_dataset_cnn_X_norm.to_array()), 0, -1)
val_X = np.moveaxis(np.array(val_dataset_cnn_X_norm.to_array()), 0, -1)
test_X = np.moveaxis(np.array(test_dataset_cnn_X_norm.to_array()), 0, -1)
tr_y = np.moveaxis(np.array(train_dataset_cnn_y_norm.to_array()), 0, -1)
val_y = np.moveaxis(np.array(val_dataset_cnn_y_norm.to_array()), 0, -1)
test_y = np.moveaxis(np.array(test_dataset_cnn_y_norm.to_array()), 0, -1)

# Convert to PyTorch tensors
tr_X_tensor = torch.tensor(tr_X, dtype=torch.float32).to('cpu')
val_X_tensor = torch.tensor(test_X, dtype=torch.float32).to('cpu')
test_X_tensor = torch.tensor(test_X, dtype=torch.float32).to('cpu')
tr_y_tensor = torch.tensor(tr_y, dtype=torch.float32).to('cpu')
val_y_tensor = torch.tensor(test_y, dtype=torch.float32).to('cpu')
test_y_tensor = torch.tensor(test_y, dtype=torch.float32).to('cpu')

tr_X_tensor = tr_X_tensor.permute(0, 3, 1, 2)
val_X_tensor = val_X_tensor.permute(0, 3, 1, 2)
test_X_tensor = test_X_tensor.permute(0, 3, 1, 2)
tr_y_tensor = tr_y_tensor.permute(0, 3, 1, 2)
val_y_tensor = val_y_tensor.permute(0, 3, 1, 2)
test_y_tensor = test_y_tensor.permute(0, 3, 1, 2)

In [6]:
def train(model, criterion, optimizer, train_inputs, train_targets, val_inputs, val_targets, scheduler, batch_size=128, epochs=200, patience=5):
    model.train()
    dataset_size = len(train_inputs)
    val_size = len(val_inputs)

    best_val_loss = float('inf')
    patience_counter = patience

    for epoch in range(epochs):
        model.train()
        permutation = torch.randperm(dataset_size)
        epoch_loss = 0
        for i in range(0, dataset_size, batch_size):
            indices = permutation[i:i + batch_size]
            batch_inputs, batch_targets = train_inputs[indices], train_targets[indices]

            optimizer.zero_grad()
            outputs = model(batch_inputs)
            loss = criterion(outputs, batch_targets)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()

        # Validation phase
        model.eval()
        val_loss = 0
        with torch.no_grad():
            for i in range(0, val_size, batch_size):
                val_batch_inputs = val_inputs[i:i + batch_size]
                val_batch_targets = val_targets[i:i + batch_size]
                val_outputs = model(val_batch_inputs)
                val_loss += criterion(val_outputs, val_batch_targets).item()

        epoch_loss /= (dataset_size // batch_size)
        val_loss /= (val_size // batch_size)
        scheduler.step(val_loss)

        print(f'Epoch {epoch+1}/{epochs}, Training Loss: {epoch_loss}, Validation Loss: {val_loss}')

        # Early stopping
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            patience_counter = patience  # Reset patience counter
            torch.save(model.state_dict(), 'best_model.pt')  # Save best model
        else:
            patience_counter -= 1

        if patience_counter == 0:
            print("Early stopping triggered.")
            break
              
train(model, criterion, optimizer, tr_X_tensor, tr_y_tensor, val_X_tensor, val_y_tensor, scheduler)

Epoch 1/200, Training Loss: 1.510502189397812, Validation Loss: 1.9903730154037476
Epoch 2/200, Training Loss: 1.4992033541202545, Validation Loss: 1.9904364347457886
Epoch 3/200, Training Loss: 1.5145569741725922, Validation Loss: 1.9903671145439148
Epoch 4/200, Training Loss: 1.50379878282547, Validation Loss: 1.9902347922325134
Epoch 5/200, Training Loss: 1.5054110884666443, Validation Loss: 1.990056037902832
Epoch 6/200, Training Loss: 1.4977684915065765, Validation Loss: 1.989726185798645
Epoch 7/200, Training Loss: 1.502721518278122, Validation Loss: 1.9890313148498535
Epoch 8/200, Training Loss: 1.501112312078476, Validation Loss: 1.9877541661262512
Epoch 9/200, Training Loss: 1.4999671876430511, Validation Loss: 1.985741913318634
Epoch 10/200, Training Loss: 1.4889505803585052, Validation Loss: 1.982936978340149
Epoch 11/200, Training Loss: 1.4956606924533844, Validation Loss: 1.9794134497642517
Epoch 12/200, Training Loss: 1.4831797778606415, Validation Loss: 1.975322306156158

In [7]:
model.eval()

with torch.no_grad():
    predictions_norm = model(test_X_tensor)

# Convert predictions to numpy array
predictions_norm_np = predictions_norm.numpy()
predictions_norm_np = predictions_norm_np.squeeze(1)

# Define dimensions and coordinates (modify based on your data)
dims = ['time', 'lat', 'lon']
coords = {
    'time': test_dataset_cnn_y['time'].values,
    'lat': test_dataset_cnn_y['lat'].values,
    'lon': test_dataset_cnn_y['lon'].values
}

# Create an xarray DataArray
predictions_norm_da = xr.DataArray(predictions_norm_np, dims=dims, coords=coords)

predictions_unnorm_da = (
    ( ( (predictions_norm_da-predictions_norm_da.mean(dim={'time','lat','lon'}))/predictions_norm_da.std(dim={'time','lat','lon'}))
     *train_dataset_cnn_y['ltg'].std(dim={'time','lat','lon'})
     )
     +train_dataset_cnn_y['ltg'].mean(dim={'time','lat','lon'})
)

predictions_unnorm_da = predictions_unnorm_da.where(predictions_unnorm_da >= 0, other = 0)
predictions_unnorm_ds = predictions_unnorm_da.to_dataset(name='ltg')

In [8]:
predictions_unnorm_ds.to_netcdf('/home/giantstep5/rjones98/meteorology/ESS569/output/cp_cnn.nc')

# Aerosol Included Run

In [2]:
ds_daily = xr.open_dataset('/home/giantstep5/rjones98/meteorology/ESS569/ai_ready/ds_jja.nc')
ds_daily = ds_daily.isel(lon=slice(0, 92), lat=slice(0, 52))
ds_daily = ds_daily.fillna(0)

dataset_cnn = ds_daily

feature_name   = ['cape', 'precipitation', 'BCCMASS', 'DUCMASS', 'OCCMASS', 'SO2CMASS', 'SO4CMASS', 'SSCMASS']
output_name    = ['ltg']

indices_train_val = np.arange(0, 488)
idx_test = np.arange(488, 736)
idx_train, idx_val = train_test_split(indices_train_val, test_size=0.25, random_state=13)
zdim, _, ydim, xdim = ds_daily[feature_name].to_array().shape

train_dataset_cnn_X = dataset_cnn[feature_name].isel(time=idx_train)
val_dataset_cnn_X = dataset_cnn[feature_name].isel(time=idx_val)
test_dataset_cnn_X  = dataset_cnn[feature_name].isel(time=idx_test)
train_dataset_cnn_y = dataset_cnn[output_name].isel(time=idx_train)
val_dataset_cnn_y = dataset_cnn[output_name].isel(time=idx_val)
test_dataset_cnn_y  = dataset_cnn[output_name].isel(time=idx_test)

In [3]:
observed_max = float(train_dataset_cnn_y.ltg.max().values)

class PenalizedMSELoss(nn.Module):
    def __init__(self, max_value, penalty_weight=10):
        super(PenalizedMSELoss, self).__init__()
        self.max_value = max_value
        self.penalty_weight = penalty_weight
        self.mse = nn.MSELoss()

    def forward(self, predictions, targets):
        # Standard MSE loss
        mse_loss = self.mse(predictions, targets)
        
        # Compute penalty for exceeding the maximum value
        penalty = torch.clamp(predictions - self.max_value, min=0) ** 2
        penalty_loss = penalty.mean()
        
        # Combine MSE loss with penalty
        total_loss = mse_loss + self.penalty_weight * penalty_loss
        return total_loss

# Initialize the penalized loss function
#criterion = PenalizedMSELoss(max_value=observed_max, penalty_weight=10)

In [4]:
seed = 13
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)

# Define the PyTorch model
class SimpleCNN(nn.Module):
    def __init__(self, zdim, ydim, xdim):
        super(SimpleCNN, self).__init__()
        self.encoder0 = self.encoder_block(zdim, 32)
        self.encoder1 = self.encoder_block(32, 16)
        self.center = self.conv_block(16, 8)
        self.decoder1 = self.decoder_block(8, 16)
        self.decoder0 = self.decoder_block(16, 32)
        self.outputs = nn.Conv2d(32, 1, kernel_size=1, padding=0)

    def conv_block(self, in_channels, out_channels):
        return nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),  # Added BatchNorm2d
            nn.ReLU(inplace=True)
        )

    def encoder_block(self, in_channels, out_channels):
        return nn.Sequential(
            self.conv_block(in_channels, out_channels),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

    def decoder_block(self, in_channels, out_channels):
        return nn.Sequential(
            nn.ConvTranspose2d(in_channels, out_channels, kernel_size=2, stride=2),
            nn.BatchNorm2d(out_channels),  # Added BatchNorm2d
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        encoder0 = self.encoder0(x)
        encoder1 = self.encoder1(encoder0)
        center = self.center(encoder1)
        decoder1 = self.decoder1(center)
        decoder0 = self.decoder0(decoder1)
        outputs = self.outputs(decoder0)
        return outputs

# Initialize model, loss function, and optimizer
zdim, ydim, xdim = 2, 208, 300  # Adjust these based on your data
model = SimpleCNN(zdim, ydim, xdim).to('cpu')
criterion = PenalizedMSELoss(max_value=observed_max, penalty_weight=5)  # Adjusted penalty_weight
optimizer = optim.Adam(model.parameters(), lr=0.00001)  # Reduced learning rate
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3)


In [5]:
def normalize(x, m, s):
    return (x - m).groupby('time.day') / s

def unnormalize(x, m, s):
    return (x * s).groupby('time.day') + m

train_dataset_cnn_X_norm = (
    train_dataset_cnn_X - train_dataset_cnn_X.mean(dim={'time', 'lat', 'lon'})
) / train_dataset_cnn_X.std(dim={'time', 'lat', 'lon'})
train_dataset_cnn_X_norm = train_dataset_cnn_X_norm.fillna(0)

val_dataset_cnn_X_norm = (
    val_dataset_cnn_X - val_dataset_cnn_X.mean(dim={'time', 'lat', 'lon'})
) / val_dataset_cnn_X.std(dim={'time', 'lat', 'lon'})
val_dataset_cnn_X_norm = val_dataset_cnn_X_norm.fillna(0)

test_dataset_cnn_X_norm = (
    test_dataset_cnn_X - test_dataset_cnn_X.mean(dim={'time', 'lat', 'lon'})
) / test_dataset_cnn_X.std(dim={'time', 'lat', 'lon'})
test_dataset_cnn_X_norm = test_dataset_cnn_X_norm.fillna(0)

train_dataset_cnn_y_norm = (
    train_dataset_cnn_y - train_dataset_cnn_y.mean(dim={'time', 'lat', 'lon'})
) / train_dataset_cnn_y.std(dim={'time', 'lat', 'lon'})
train_dataset_cnn_y_norm = train_dataset_cnn_y_norm.fillna(0)

val_dataset_cnn_y_norm = (
    val_dataset_cnn_y - val_dataset_cnn_y.mean(dim={'time', 'lat', 'lon'})
) / val_dataset_cnn_y.std(dim={'time', 'lat', 'lon'})
val_dataset_cnn_y_norm = val_dataset_cnn_y_norm.fillna(0)

test_dataset_cnn_y_norm = (
    test_dataset_cnn_y - test_dataset_cnn_y.mean(dim={'time', 'lat', 'lon'})
) / test_dataset_cnn_y.std(dim={'time', 'lat', 'lon'})
test_dataset_cnn_y_norm = test_dataset_cnn_y_norm.fillna(0)

# Convert to numpy arrays and move axis
tr_X = np.moveaxis(np.array(train_dataset_cnn_X_norm.to_array()), 0, -1)
val_X = np.moveaxis(np.array(val_dataset_cnn_X_norm.to_array()), 0, -1)
test_X = np.moveaxis(np.array(test_dataset_cnn_X_norm.to_array()), 0, -1)
tr_y = np.moveaxis(np.array(train_dataset_cnn_y_norm.to_array()), 0, -1)
val_y = np.moveaxis(np.array(val_dataset_cnn_y_norm.to_array()), 0, -1)
test_y = np.moveaxis(np.array(test_dataset_cnn_y_norm.to_array()), 0, -1)

# Convert to PyTorch tensors
tr_X_tensor = torch.tensor(tr_X, dtype=torch.float32).to('cpu')
val_X_tensor = torch.tensor(test_X, dtype=torch.float32).to('cpu')
test_X_tensor = torch.tensor(test_X, dtype=torch.float32).to('cpu')
tr_y_tensor = torch.tensor(tr_y, dtype=torch.float32).to('cpu')
val_y_tensor = torch.tensor(test_y, dtype=torch.float32).to('cpu')
test_y_tensor = torch.tensor(test_y, dtype=torch.float32).to('cpu')

tr_X_tensor = tr_X_tensor.permute(0, 3, 1, 2)
val_X_tensor = val_X_tensor.permute(0, 3, 1, 2)
test_X_tensor = test_X_tensor.permute(0, 3, 1, 2)
tr_y_tensor = tr_y_tensor.permute(0, 3, 1, 2)
val_y_tensor = val_y_tensor.permute(0, 3, 1, 2)
test_y_tensor = test_y_tensor.permute(0, 3, 1, 2)

In [6]:
def train(model, criterion, optimizer, train_inputs, train_targets, val_inputs, val_targets, scheduler, batch_size=128, epochs=200, patience=5):
    model.train()
    dataset_size = len(train_inputs)
    val_size = len(val_inputs)

    best_val_loss = float('inf')
    patience_counter = patience

    for epoch in range(epochs):
        model.train()
        permutation = torch.randperm(dataset_size)
        epoch_loss = 0
        for i in range(0, dataset_size, batch_size):
            indices = permutation[i:i + batch_size]
            batch_inputs, batch_targets = train_inputs[indices], train_targets[indices]

            optimizer.zero_grad()
            outputs = model(batch_inputs)
            loss = criterion(outputs, batch_targets)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()

        # Validation phase
        model.eval()
        val_loss = 0
        with torch.no_grad():
            for i in range(0, val_size, batch_size):
                val_batch_inputs = val_inputs[i:i + batch_size]
                val_batch_targets = val_targets[i:i + batch_size]
                val_outputs = model(val_batch_inputs)
                val_loss += criterion(val_outputs, val_batch_targets).item()

        epoch_loss /= (dataset_size // batch_size)
        val_loss /= (val_size // batch_size)
        scheduler.step(val_loss)

        print(f'Epoch {epoch+1}/{epochs}, Training Loss: {epoch_loss}, Validation Loss: {val_loss}')

        # Early stopping
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            patience_counter = patience  # Reset patience counter
            torch.save(model.state_dict(), 'best_model.pt')  # Save best model
        else:
            patience_counter -= 1

        if patience_counter == 0:
            print("Early stopping triggered.")
            break
              
train(model, criterion, optimizer, tr_X_tensor, tr_y_tensor, val_X_tensor, val_y_tensor, scheduler)

Epoch 1/200, Training Loss: 1.510502189397812, Validation Loss: 1.9903730154037476
Epoch 2/200, Training Loss: 1.4992033541202545, Validation Loss: 1.9904364347457886
Epoch 3/200, Training Loss: 1.5145569741725922, Validation Loss: 1.9903671145439148
Epoch 4/200, Training Loss: 1.50379878282547, Validation Loss: 1.9902347922325134
Epoch 5/200, Training Loss: 1.5054110884666443, Validation Loss: 1.990056037902832
Epoch 6/200, Training Loss: 1.4977684915065765, Validation Loss: 1.989726185798645
Epoch 7/200, Training Loss: 1.502721518278122, Validation Loss: 1.9890313148498535
Epoch 8/200, Training Loss: 1.501112312078476, Validation Loss: 1.9877541661262512
Epoch 9/200, Training Loss: 1.4999671876430511, Validation Loss: 1.985741913318634
Epoch 10/200, Training Loss: 1.4889505803585052, Validation Loss: 1.982936978340149
Epoch 11/200, Training Loss: 1.4956606924533844, Validation Loss: 1.9794134497642517
Epoch 12/200, Training Loss: 1.4831797778606415, Validation Loss: 1.975322306156158

In [7]:
model.eval()

with torch.no_grad():
    predictions_norm = model(test_X_tensor)

# Convert predictions to numpy array
predictions_norm_np = predictions_norm.numpy()
predictions_norm_np = predictions_norm_np.squeeze(1)

# Define dimensions and coordinates (modify based on your data)
dims = ['time', 'lat', 'lon']
coords = {
    'time': test_dataset_cnn_y['time'].values,
    'lat': test_dataset_cnn_y['lat'].values,
    'lon': test_dataset_cnn_y['lon'].values
}

# Create an xarray DataArray
predictions_norm_da = xr.DataArray(predictions_norm_np, dims=dims, coords=coords)

predictions_unnorm_da = (
    ( ( (predictions_norm_da-predictions_norm_da.mean(dim={'time','lat','lon'}))/predictions_norm_da.std(dim={'time','lat','lon'}))
     *train_dataset_cnn_y['ltg'].std(dim={'time','lat','lon'})
     )
     +train_dataset_cnn_y['ltg'].mean(dim={'time','lat','lon'})
)

predictions_unnorm_da = predictions_unnorm_da.where(predictions_unnorm_da >= 0, other = 0)
predictions_unnorm_ds = predictions_unnorm_da.to_dataset(name='ltg')

In [9]:
predictions_unnorm_ds.to_netcdf('/home/giantstep5/rjones98/meteorology/ESS569/output/cp_aer_cnn.nc')